## Visualise model detections
Use fiftyone to visualise the detections of a model.

In [ ]:
from pathlib import Path

import torch
import fiftyone as fo
from torchvision.ops import box_convert
from tqdm import tqdm

from ssd import SSD
from ssd.data import LetterboxTransform, SSDDataset

### Define constants

In [ ]:
MODEL_FILE = Path(
    "/mnt/data/code/ssd/runs/49ffe3fa-f06f-4b7d-9677-2fa87f112544/best.pt"
)

DATASET_PATH = Path("/mnt/data/datasets/object_detection/coco")
DATASET_NAME = "coco"

IMAGE_WIDTH = 300
IMAGE_HEIGHT = 300
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

### Load the model

In [ ]:
model = SSD.load(MODEL_FILE)

### Create the dataset object

In [ ]:
transform = LetterboxTransform(IMAGE_WIDTH, IMAGE_HEIGHT)
dataset = SSDDataset(
    DATASET_PATH / "images/val",
    DATASET_PATH / "labels/val",
    80,
    transform,
    None,
    DEVICE,
    DTYPE,
)

### Run inference and add samples to fiftyone

In [ ]:
fo_dataset = fo.Dataset(DATASET_NAME)

for idx, (image_file, label_file) in tqdm(enumerate(dataset.samples)):
    sample = fo.Sample(image_file)

    # Add label boxes to the sample
    objects = SSDDataset.read_label_file(label_file, DEVICE, DTYPE)
    object_class_ids = objects.class_ids.cpu().tolist()
    object_boxes = box_convert(objects.boxes, "cxcywh", "xywh").cpu().tolist()
    object_detections = fo.Detections(
        detections=[
            fo.Detection(label=f"{class_id}", bounding_box=box)
            for class_id, box in zip(object_class_ids, object_boxes, strict=True)
        ]
    )
    sample["ground_truth"] = object_detections

    # Run inference on the frame
    predictions = model.infer(image_file, 0.3, 200)
    prediction_class_ids = predictions.class_ids.cpu().tolist()
    prediction_boxes = box_convert(predictions.boxes, "cxcywh", "xywh").cpu().tolist()
    prediction_scores = predictions.scores.cpu().tolist()
    prediction_detections = fo.Detections(
        detections=[
            fo.Detection(label=f"{class_id}", bounding_box=box, confidence=score)
            for class_id, box, score in zip(
                prediction_class_ids, prediction_boxes, prediction_scores, strict=True
            )
        ]
    )
    sample["predictions"] = prediction_detections
    fo_dataset.add_sample(sample)

In [ ]:
session = fo.launch_app(fo_dataset)